<div style="
  border-left: 6px solid #bfc3c7;
  background: #f8fafc;
  padding: 14px 18px;
  border-radius: 10px;
  margin: 12px 0;
  font-family: Arial, sans-serif;
  box-shadow: 0 3px 8px rgba(0,0,0,0.05);
">

  <h3 style="margin:0; color:#6b7280;">🥈 Silver Layer</h3>

  <p style="margin:6px 0 0; color:#444;">
    Cleaned and standardized data. Deduplicated, validated,
    and structured for analytics.
  </p>

</div>


lets import the most important Libraries

In [ ]:
import pandas as pd


load in bronze dataframe as a copy for the new silver df

In [18]:
silver_df = pd.read_csv('../Data/bronze.csv').copy()


then lets normalize the dataframe

In [19]:
# 1. Remove first row with units (EUR/MWh)
silver_df = silver_df.iloc[1:].reset_index(drop=True)

# 2. Clean Date column: remove [ ]
silver_df["Date (CET)"] = (
    silver_df["Date (CET)"]
    .str.replace(r"[\[\]]", "", regex=True)
)

# 3. Convert to datetime
silver_df["Date (CET)"] = pd.to_datetime(
    silver_df["Date (CET)"],
    format="%d/%m/%Y %H:%M"
)

# 4. Convert all price columns to float
price_columns = silver_df.columns.drop("Date (CET)")

silver_df[price_columns] = silver_df[price_columns].apply(
    pd.to_numeric,
    errors="coerce"
)

silver_df.columns = (
    silver_df.columns
    .str.lower()
    .str.replace(r"[()\-]", "", regex=True)
    .str.replace(" ", "_")
)

silver_df.head()

# 5. Rename date column
silver_df = silver_df.rename(columns={"Date (CET)": "datetime_cet"})



silver_df.head()

,date_cet,belgium_be,denmark_dk,germany_de,great_britain_gb,great_britain_gb__epex,great_britain_gb__nordpool,netherlands_nl,norway_no
0,2026-02-10 14:00:00,103.19,118.02,116.22,108.68,108.28,108.84,109.12,113.17
1,2026-02-10 15:00:00,112.55,130.20,128.12,111.38,111.38,111.38,119.64,124.07
2,2026-02-10 16:00:00,110.50,131.15,127.66,113.96,114.71,113.67,118.42,124.14
3,2026-02-10 17:00:00,119.26,138.66,135.24,120.55,120.56,120.55,126.72,131.54
4,2026-02-10 18:00:00,121.80,136.04,131.71,123.04,123.09,123.03,126.35,130.60


Lets get a overview of the data now

In [20]:
silver_df.describe()


,date_cet,belgium_be,denmark_dk,germany_de,great_britain_gb,great_britain_gb__epex,great_britain_gb__nordpool,netherlands_nl,norway_no
count,32,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,2026-02-11 05:30:00,94.548750,115.970625,115.265625,101.466250,102.473437,101.100000,104.174063,113.303750
min,2026-02-10 14:00:00,72.230000,94.830000,94.680000,82.410000,86.860000,80.620000,82.880000,97.550000
25%,2026-02-10 21:45:00,85.280000,105.277500,104.792500,93.030000,94.760000,92.332500,95.367500,103.205000
50%,2026-02-11 05:30:00,93.445000,117.030000,116.460000,100.350000,100.730000,99.525000,103.820000,113.310000
75%,2026-02-11 13:15:00,103.945000,128.010000,127.540000,109.862500,109.832500,109.352500,114.315000,124.087500
max,2026-02-11 21:00:00,121.800000,138.660000,137.070000,123.040000,123.090000,123.030000,126.720000,132.770000
std,NaN,13.984296,13.706524,13.122081,10.804858,10.103763,11.147856,13.073547,11.261233


now lets save the new silver df

In [21]:
silver_df.to_csv("../Data/silver.csv", index=False)